In [14]:
from biobert_embedding.embedding import BiobertEmbedding
from scipy.spatial import distance
import pandas as pd
import os

In [15]:
ner_df = pd.read_pickle("data/ner_df.p")


In [17]:
null_index_list = []
for i in ner_df.itertuples():
    if len(i.ner) == 0:
        null_index_list.append(i.Index)
ner_df.drop(null_index_list, inplace=True)

In [23]:
print(len(ner_df))

In [ ]:
def handle_truncate_list(key, sentence_vector, my_dict, max_items_in_list =3):
    if key in my_dict: 
        
        # we must do some logic to remove the most similar item
        if len(my_dict[key]) == max_items_in_list: 
            my_list  = my_dict[key]
            similarity_value = 0.00
            index = -1
            for i, vector_to_compare in enumerate(my_list):
                cosine_sim = 1 - distance.cosine(sentence_vector, vector_to_compare)
                if cosine_sim > similarity_value:
                    index = i
                    similarity_value = cosine_sim
            if index > -1:
                my_dict[key].pop(index)
    else:
        my_dict[key] = list()
    
    my_dict[key].append(sentence_vector)

In [ ]:
count = 0
biobert = BiobertEmbedding("/home/ubuntu/biobertmodel")

new_biobert_dict = {}
with open("/home/ubuntu/bad_vectors.txt", "w") as file1:
    # Writing data to a file
    sentence_vector = ""
    num_exceptions = 0
    for i in ner_df.itertuples():
        # print(i)
        
        # we should only look for ids that are relevant
        if i.SUBJECT_ID not in ner_fasttext:
            continue
            
        if count % 1000 == 0:
            print(f"processed {count} with exceptions: {num_exceptions}")
        if i.preprocessed_text is not None and len(i.preprocessed_text[0]) > 0:
            try:
                # print(i.preprocessed_text[0])
                sentence_vector = biobert.sentence_vector(i.preprocessed_text[0])
                handle_truncate_list(i.SUBJECT_ID, sentence_vector, new_biobert_dict)
                #if i.SUBJECT_ID not in new_biobert_dict:
                 #   new_biobert_dict[i.SUBJECT_ID] = list(sentence_vector)            
                #else:
                 #   new_biobert_dict[i.SUBJECT_ID].append(sentence_vector) 
            except Exception as e:
                print(str(e))
                num_exceptions += 1
                file1.write(f"error on subject: {i.SUBJECT_ID}    {i.preprocessed_text[0]}" + "\n")
                unique_keywords = dict()
                to_sentence = list()
                for list_of_tuples in i.ner:
                    for a_tuple in list_of_tuples:
                        if a_tuple[0] not in unique_keywords:
                            unique_keywords[a_tuple[0]] = a_tuple[1]
                            to_sentence.append(a_tuple[0])
                            to_sentence.append(a_tuple[1])
                
                to_sentence = [word.strip() for word in to_sentence]
                sentence_vector = biobert.sentence_vector(' '.join(to_sentence))
                
                handle_truncate_list(i.SUBJECT_ID, sentence_vector, new_biobert_dict)
                
                #if i.SUBJECT_ID not in new_biobert_dict:
                 #   new_biobert_dict[i.SUBJECT_ID] = list(' '.join(to_sentence))            
                #else:
                #    new_biobert_dict[i.SUBJECT_ID].append(' '.join(to_sentence)) 
                    
        count += 1
    



In [ ]:
pd.to_pickle(new_biobert_dict, "data/biobert_test_data.pkl")

In [ ]:
len(list(new_biobert_dict.keys()))